In [1]:
!pip -q install crewai langchain-groq duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!pip show crewai serpapi

Name: crewai
Version: 0.30.11
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


## API keys

In [3]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

SERP_API_KEY = userdata.get('SERP_API_KEY')
if SERP_API_KEY is None:
    raise ValueError("SERP_API_KEY secret not found in Colab. Please add it in the secrets section.")

os.environ["SERP_API_KEY"] = SERP_API_KEY


In [4]:
import os
from google.colab import userdata
from crewai import Crew, Agent, Task, Process
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
import json
import requests

In [5]:
# Groq model
GROQ_LLM = ChatGroq(model="llama3-70b-8192")

In [6]:
def fetch_news(query, api_key=SERP_API_KEY):
    params = {
        "engine": "google",
        "q": query,
        "tbm": "nws",
        "api_key": api_key
    }
    response = requests.get("https://google.serper.dev/search", params=params)
    return response.json()

## Define the agents


In [7]:
# Define the agents
class NewsAgents():
    def make_search_agent(self):
        return Agent(
            role='News Search Agent',
            goal="""Search for news articles using query thhat is provided to you.""",
            backstory="""You are skilled at finding relevant news articles on the internet.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            tools=[DuckDuckGoSearchRun()],
        )

    def make_full_text_agent(self):
        return Agent(
            role='Full Text Agent',
            goal="Fetch full text from news articles.",
            backstory="You are skilled in retrieving the full content of articles from given links.",
            llm=GROQ_LLM,
            verbose=True
        )

    def make_summary_agent(self):
        return Agent(
            role='Article Summary Agent',
            goal="""Summarize the news articles i very precise manner so that user can get only relevent and important information.""",
            backstory="""You master in making summary from news articals.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

    def make_keyword_agent(self):
        return Agent(
            role='Keyword Extraction Agent',
            goal="""Extract key words from the news articles.""",
            backstory="""You are adept at identifying key words from news articals.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

## Define the tasks

In [8]:
class NewsTasks():
    def search_news(self, query):
        return Task(
            description=f"""Search for news articles with the query: {query}.""",
            expected_output="""A list of news articles with details like date, region, source, article summary, and keywords.""",
            output_file=f"news_articles.json",
            agent=search_agent
        )

    def fetch_full_text(self, article_link):
        return Task(
            description=f"Fetch the full text of the article from the following link: {article_link}",
            agent=full_text_agent
        )

    def summarize_article(self, article):
        return Task(
            description=f"""Summarize the following news article: {article}.""",
            expected_output="""A summary of the article.""",
            context=[search_news],
            agent=summary_agent
        )

    def extract_keywords(self, article):
        return Task(
            description=f"""Extract key words from the following news article: {article}.""",
            expected_output="""A list of key words from the article.""",
            context=[search_news],
            agent=keyword_agent
        )

In [9]:
# Example query
QUERY = "latest tech news in 2022"

In [10]:
# Instantiate agents and tasks
agents = NewsAgents()
tasks = NewsTasks()

search_agent = agents.make_search_agent()
full_text_agent = agents.make_full_text_agent()
summary_agent = agents.make_summary_agent()
keyword_agent = agents.make_keyword_agent()

search_news = tasks.search_news(QUERY)

In [11]:
def process_news_articles(news_data):
    articles = news_data.get('news_results', [])
    formatted_articles = []
    for article in articles:
        date = article.get('date')
        region = article.get('region', 'N/A')
        source = article.get('source')
        title = article.get('title')
        link = article.get('link')

        full_text_task = tasks.fetch_full_text(link)
        full_text = full_text_task.agent.complete_task(link)

        summary_task = tasks.summarize_article(full_text)
        summary = summary_task.agent.complete_task(full_text)

        keywords_task = tasks.extract_keywords(full_text)
        keywords = keywords_task.agent.complete_task(full_text)

        formatted_articles.append({
            "date": date,
            "region": region,
            "source": source,
            "article": full_text,
            "article_summary": summary,
            "keywords": keywords
        })

    return formatted_articles

In [12]:
news_data = fetch_news(QUERY)

In [13]:
# Process the news articles
formatted_articles = process_news_articles(news_data)

In [14]:
for idx, article in enumerate(formatted_articles, start=1):
    print(f"**Article {idx}:**")
    print(f"Date: {article['date']}")
    print(f"Region: {article['region']}")
    print(f"Source: {article['source']}")
  # print(f"Article: {article['article']}")
    print(f"Article Summary: {article['article_summary']}")
    print(f"Keywords: {article['keywords']}")
    print("\n")

# with open('formatted_articles.json', 'w') as f:
#     json.dump(formatted_articles, f, indent=4)

## PrintSchema

In [15]:
def print_agent_output(agent_output, agent_name):
    if isinstance(agent_output, str):
        print(f"{agent_name}: {agent_output}")
    elif isinstance(agent_output, list):
        for item in agent_output:
            print(f"{agent_name}: {item}")
    else:
        print(f"{agent_name}: Unknown output format")

## Crew and kickoff





In [16]:
# Instantiate your crew with a sequential process
crew = Crew(
    agents=[search_agent, summary_agent, keyword_agent],
    tasks=[search_news],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [17]:
results = crew.kickoff()

print("Crew Work Results:")
print(results)

print(crew.usage_metrics)

 [DEBUG]: == Working Agent: News Search Agent
 [INFO]: == Starting Task: Search for news articles with the query: latest tech news in 2022.


> Entering new CrewAgentExecutor chain...
I need to search for news articles using the query "latest tech news in 2022".

Action: duckduckgo_search
Action Input: {"q": "latest tech news in 2022"} 

I encountered an error while trying to use the tool. This was the error: DuckDuckGoSearchRun._run() got an unexpected keyword argument 'q'.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

MasterCrew Agent: (AgentAction(tool='duckduckgo_search', tool_input='{"q": "latest tech news in 2022"}', log='I need to search for news articles using the query "latest tech news in 2022".\n\nAction: duckduckgo_search\nAction Input: {"q": "latest tech news in 2022"}'), "\nI encountered an error while trying to use the tool. This was the

# FULL CODE

In [76]:
import os
from google.colab import userdata
from crewai import Crew, Agent, Task, Process
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
import json
import requests

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

SERP_API_KEY = userdata.get('SERP_API_KEY')
if SERP_API_KEY is None:
    raise ValueError("SERP_API_KEY secret not found in Colab. Please add it in the secrets section.")

os.environ["SERP_API_KEY"] = SERP_API_KEY

# Groq language model
GROQ_LLM = ChatGroq(model="llama3-70b-8192")

def fetch_news(query, api_key=SERP_API_KEY):
    params = {
        "engine": "google",
        "q": query,
        "tbm": "nws",
        "api_key": api_key
    }
    response = requests.get("https://google.serper.dev/search", params=params)
    return response.json()

# Define the agents
class NewsAgents():
    def make_search_agent(self):
        return Agent(
            role='News Search Agent',
            goal="Search for news articles using the query provided to you.",
            backstory="You are skilled at finding relevant news articles on the internet.",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            tools=[DuckDuckGoSearchRun()],
        )

    def make_full_text_agent(self):
        return Agent(
            role='Full Text Agent',
            goal="Fetch full text from news articles.",
            backstory="You are skilled in retrieving the full content of articles from given links.",
            llm=GROQ_LLM,
            verbose=True
        )

    def make_summary_agent(self):
        return Agent(
            role='Article Summary Agent',
            goal="Summarize the news articles in a very precise manner so that users can get only relevant and important information.",
            backstory="You master in making summaries from news articles.",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

    def make_keyword_agent(self):
        return Agent(
            role='Keyword Extraction Agent',
            goal="Extract keywords from the news articles.",
            backstory="You are adept at identifying keywords from news articles.",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

# Define the tasks
class NewsTasks():
    def search_news(self, query):
        return Task(
            description=f"Search for news articles with the query: {query}.",
            expected_output="A list of news articles with details like date, region, source, article summary, and keywords.",
            output_file="news_articles.json",
            agent=search_agent
        )

    def fetch_full_text(self, article_link):
        return Task(
            description=f"Fetch the full text of the article from the following link: {article_link}",
            agent=full_text_agent
        )

    def summarize_article(self, article):
        return Task(
            description=f"Summarize the following news article: {article}.",
            expected_output="A summary of the article.",
            context=[search_news],
            agent=summary_agent
        )

    def extract_keywords(self, article):
        return Task(
            description=f"Extract keywords from the following news article: {article}.",
            expected_output="A list of keywords from the article.",
            context=[search_news],
            agent=keyword_agent
        )

# Example query
QUERY = "latest tech news in 2020"

# Instantiate agents and tasks
agents = NewsAgents()
tasks = NewsTasks()

search_agent = agents.make_search_agent()
full_text_agent = agents.make_full_text_agent()
summary_agent = agents.make_summary_agent()
keyword_agent = agents.make_keyword_agent()

search_news = tasks.search_news(QUERY)

def process_news_articles(news_data):
    articles = news_data.get('news_results', [])
    formatted_articles = []
    for article in articles:
        date = article.get('date')
        region = article.get('region', 'N/A')
        source = article.get('source')
        title = article.get('title')
        link = article.get('link')

        # Fetch full text of the article
        full_text_task = tasks.fetch_full_text(link)
        full_text = full_text_task.agent.complete_task(link)

        # Summarize the full text
        summary_task = tasks.summarize_article(full_text)
        summary = summary_task.agent.complete_task(full_text)

        # Extract keywords from the full text
        keywords_task = tasks.extract_keywords(full_text)
        keywords = keywords_task.agent.complete_task(full_text)

        formatted_articles.append({
            "date": date,
            "region": region,
            "source": source,
            "article": full_text,
            "article_summary": summary,
            "keywords": keywords
        })

    return formatted_articles

# Fetch news articles using SerpAPI
news_data = fetch_news(QUERY)

# Process the news articles
formatted_articles = process_news_articles(news_data)

for idx, article in enumerate(formatted_articles, start=1):
    print(f"**Article {idx}:**")
    print(f"Date: {article['date']}")
    print(f"Region: {article['region']}")
    print(f"Source: {article['source']}")
    print(f"Article: {article['article']}")
    print(f"Article Summary: {article['article_summary']}")
    print(f"Keywords: {article['keywords']}")
    print("\n")

# Save the results to a file named after the query
output_file_name = QUERY.replace(" ", "_") + ".txt"
with open(output_file_name, 'w') as f:
    for idx, article in enumerate(formatted_articles, start=1):
        f.write(f"**Article {idx}:**\n")
        f.write(f"Date: {article['date']}\n")
        f.write(f"Region: {article['region']}\n")
        f.write(f"Source: {article['source']}\n")
        f.write(f"Article: {article['article']}\n")
        f.write(f"Article Summary: {article['article_summary']}\n")
        f.write(f"Keywords: {article['keywords']}\n\n")

def print_agent_output(agent_output, agent_name):
    if isinstance(agent_output, str):
        print(f"{agent_name}: {agent_output}")
    elif isinstance(agent_output, list):
        for item in agent_output:
            print(f"{agent_name}: {item}")
    else:
        print(f"{agent_name}: Unknown output format")

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[search_agent, full_text_agent, summary_agent, keyword_agent],
    tasks=[search_news],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

results = crew.kickoff()

print("Crew Work Results:")
print(results)

print(crew.usage_metrics)


 [DEBUG]: == Working Agent: News Search Agent
 [INFO]: == Starting Task: Search for news articles with the query: latest tech news in 2020.


> Entering new CrewAgentExecutor chain...
I need to find the latest tech news from 2020. To do this, I'll use the duckduckgo_search tool to search for relevant news articles.

Action: duckduckgo_search
Action Input: {"q": "latest tech news 2020"}
 

I encountered an error while trying to use the tool. This was the error: DuckDuckGoSearchRun._run() got an unexpected keyword argument 'q'.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

MasterCrew Agent: (AgentAction(tool='duckduckgo_search', tool_input='{"q": "latest tech news 2020"}\n', log='I need to find the latest tech news from 2020. To do this, I\'ll use the duckduckgo_search tool to search for relevant news articles.\n\nAction: duckduckgo_search\nAction Input: